In [22]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import requests

store_list    = ["ricardoeletro", "magazineluiza"]

dataset_pages = f"data/heuristic_pages/*/*"
dataset_links = f"data/heusristic_links/heusristic_links/*"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dataset_data = [(url, pathlib.Path(url).stem.split('_')[0] ) for url in glob.glob(dataset_links) if len(url.split('_'))>3]

In [ ]:
datasets = []

dataset_idx = 0
data_raw_s = pd.read_csv(dataset_data[dataset_idx][0])
data_raw_s = data_raw_s.dropna().drop_duplicates()
data_raw_s['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw_s.shape[0])

for dataset_idx in range(1, len(dataset_data)):
    data_raw = pd.read_csv(dataset_data[dataset_idx][0])
    data_raw = data_raw.dropna().drop_duplicates()
    data_raw['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw.shape[0])
    data_raw_s = pd.merge(data_raw_s, data_raw, how='outer')
    #datasets.append(data_raw)

In [58]:
x_pages = [f'data/heuristic_pages/{store}/page_{p_id}.html' for (store, p_id) in zip(data_raw_s['store'], data_raw_s['id'])]
y = data_raw_s['label'].values
u_class = data_raw_s['store']
l = data_raw_s['links'].values

In [22]:
X, idx_arr = pages_to_matrix(x_pages)
y = y[idx_arr]
u_class = u_class[idx_arr]
l = l[idx_arr]
X.shape, y.shape, u_class.shape, l.shape


Invalid Links 4


In [63]:
df = pd.DataFrame()
df['store'] = u_class
df['link'] = l
df['text'] = X
df['label'] = y
df.to_csv("text_store_label.csv", index=False)

### Test

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

In [54]:
df = pd.read_csv('text_store_label.csv')
df = df.fillna(" ")
u_class = df['store'].values
l  = df['link'].values
X  = df['text'].values
y  = df['label'].values

pass

In [55]:
clss = [GaussianNB(), MultinomialNB(), KNeighborsClassifier(), SVC(gamma='auto'), 
        RandomForestClassifier(n_estimators=200), GradientBoostingClassifier(n_estimators=200)]

# T

In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [58]:
CV = CountVectorizer()
    
for cls in clss:

    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        #('tfidf', TfidfTransformer()),
        ('tranf', DenseTransformer()),
        ('clf', cls),
    ])
     
    text_clf.fit(X_train, y_train)  
    predicted = text_clf.predict(X_test)
        
    print(cls.__class__.__name__, np.mean(predicted == y_test))

GaussianNB 0.950381679389313
MultinomialNB 0.9580152671755725
KNeighborsClassifier 0.9847328244274809
SVC 0.9713740458015268
RandomForestClassifier 0.9904580152671756
GradientBoostingClassifier 0.9904580152671756


In [59]:
CV = CountVectorizer()
    
for cls in clss:
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('tranf', DenseTransformer()),
        ('clf', cls),
    ])
    
    
    text_clf.fit(X_train, y_train)  
    predicted = text_clf.predict(X_test)
        
    print(cls.__class__.__name__, np.mean(predicted == y_test))

GaussianNB 0.9751908396946565
MultinomialNB 0.9732824427480916
KNeighborsClassifier 0.982824427480916
SVC 0.8187022900763359
RandomForestClassifier 0.9885496183206107
GradientBoostingClassifier 0.9904580152671756


# Leave One Out

In [62]:
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()

In [63]:
acc_results = []
header = [cls.__class__.__name__ for cls in clss]
for train_index, test_index in tqdm(logo.split(X, y, u_class)):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    stores_results = []
    
    for cls in clss:
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('tranf', DenseTransformer()),
            ('clf', cls),
        ])


        text_clf.fit(X_train, y_train)  
        predicted = text_clf.predict(X_test)
        acc = np.mean(predicted == y_test)
        stores_results.append(acc)
    acc_results.append(stores_results)
acc_results = pd.DataFrame(acc_results, columns=header)
acc_results.describe()

,GaussianNB,MultinomialNB,KNeighborsClassifier,SVC,RandomForestClassifier,GradientBoostingClassifier
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.747006,0.879460,0.750223,0.750118,0.871736,0.914445
std,0.221189,0.142182,0.116524,0.149049,0.154113,0.154761
min,0.353846,0.547554,0.550000,0.516667,0.547554,0.547554
25%,0.591898,0.859021,0.684528,0.631680,0.775184,0.947577
50%,0.816533,0.939256,0.758200,0.781136,0.945109,0.989600
75%,0.916903,0.966813,0.827767,0.860122,0.989439,0.995106
max,0.990654,0.990654,0.924765,0.924765,0.996226,1.000000


In [65]:
acc_results = []
header = [cls.__class__.__name__ for cls in clss]
for train_index, test_index in tqdm(logo.split(X, y, u_class)):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    stores_results = []
    
    for cls in clss:
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            #('tfidf', TfidfTransformer()),
            ('tranf', DenseTransformer()),
            ('clf', cls),
        ])


        text_clf.fit(X_train, y_train)  
        predicted = text_clf.predict(X_test)
        acc = np.mean(predicted == y_test)
        stores_results.append(acc)
    acc_results.append(stores_results)
acc_results = pd.DataFrame(acc_results, columns=header)
acc_results.describe()

,GaussianNB,MultinomialNB,KNeighborsClassifier,SVC,RandomForestClassifier,GradientBoostingClassifier
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.761671,0.877777,0.790874,0.804606,0.889923,0.915698
std,0.244803,0.145558,0.152899,0.174059,0.164545,0.155355
min,0.323077,0.547554,0.542120,0.512228,0.547554,0.547554
25%,0.595227,0.885123,0.725406,0.719502,0.776640,0.950711
50%,0.838847,0.922230,0.808087,0.881512,0.992272,0.989600
75%,0.959287,0.953008,0.880353,0.934316,0.996002,0.995106
max,0.995327,1.000000,1.000000,0.996226,1.000000,1.000000


## Final CLF

In [66]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    #('tfidf', TfidfTransformer()),
    ('tranf', DenseTransformer()),
    ('clf', GradientBoostingClassifier),
])


text_clf.fit(X_train, y_train)  
predicted = text_clf.predict(X_test)
acc = np.mean(predicted == y_test)
stores_results.append(acc)
acc_results.append(stores_results)

sklearn.ensemble.gradient_boosting.GradientBoostingClassifier

# Link Classifier

In [75]:
import re

In [119]:
link_x = [re.sub(r'https.*.com', '', li).replace(r'.br', '') for li in l]


In [120]:
re_letter = re.compile('[^a-zA-Z ]')


re_letter_s = [re_letter.sub(' ',lx) for lx in link_x]
re_letter_s = [re.sub(r'\b\w{1,3}\b', '', re_l)for re_l in re_letter_s]

In [127]:
acc_results = []
header = [cls.__class__.__name__ for cls in clss]
for train_index, test_index in tqdm(logo.split(l, y, u_class)):

    X_train, X_test = l[train_index], l[test_index]
    y_train, y_test = y[train_index], y[test_index]
    count_vect = CountVectorizer()
    
    X_train = count_vect.fit_transform(X_train)
    X_test  = count_vect.transform(X_test)
    stores_results = []
    
    for cls in clss:
        
        text_clf = Pipeline([
            #('tfidf', TfidfTransformer()),
            ('tranf', DenseTransformer()),
            ('clf', cls),
        ])
        


        text_clf.fit(X_train, y_train)  
        predicted = text_clf.predict(X_test)
        acc = np.mean(predicted == y_test)
        stores_results.append(acc)
    acc_results.append(stores_results)
acc_results = pd.DataFrame(acc_results, columns=header)
acc_results.describe()

,GaussianNB,MultinomialNB,KNeighborsClassifier,SVC,RandomForestClassifier,GradientBoostingClassifier
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.747184,0.743445,0.750544,0.741398,0.769654,0.664245
std,0.203964,0.321598,0.166346,0.143948,0.171269,0.268284
min,0.430518,0.192935,0.468665,0.516667,0.476839,0.295775
25%,0.609456,0.512032,0.689229,0.631680,0.649306,0.453826
50%,0.742134,0.922354,0.784992,0.751184,0.756794,0.683968
75%,0.908333,0.945881,0.880839,0.840573,0.935275,0.918485
max,1.000000,0.990654,0.934169,0.924765,0.957746,0.985149
